In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.0/273.0 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [12]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [13]:
pdfreader = PdfReader('/content/1.12 Eyewitness Identification 2017.pdf')

In [14]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [15]:
raw_text

"  \nNORWOOD  POLICE DEPARTMENT  1 (May 26th 2017 ) \n  EYEWITNESS IDENTIF ICATION  \nPOLICY & PROCEDURE NO.  \n1.12  \nISSUE  \nDATE: 11/21/13  \n \nEFFECTIVE  \nDATE: 11/21/13  \nMASSACHUSETTS POLICE \nACCREDITATION STANDARDS  \nREFERENCED:  1.2.3, 42.2.3(e)   \nREVISION  \nDATE: 05/26/17  \nGENERAL CONSIDERATIONS  \n \nEyewitness identifi cation  procedures  must be conducted in a fair, objective, and non -\nsuggestive manner.  When identification procedures conducted by the police are \nunnecessarily suggestive, and conducive to irreparable mistaken identification, it is a \nviolation of due proces s that may result in a wrongful conviction , or the exclusion of \nevidence .1  Therefore, the identification of criminal offenders must be approached with \nextreme caution  to ensure the proper administration of justice, and to prevent the court \nfrom excluding  or limit ing eyewitness evidence if it determines that police methods were \nunnecessarily suggestive.   \n \nThe identifica

In [16]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [17]:
len(texts)

53

In [18]:
embeddings = OpenAIEmbeddings()

In [19]:
document_search = FAISS.from_texts(texts, embeddings)

In [20]:
document_search

In [21]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [22]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [23]:
query = "How long can I wait to conduct a show up?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" A show-up should not be conducted more than two hours after the witness's observation of the offender."

In [24]:
query = "How will mugshots be taken?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Officers will only use one photograph of each individual and will not refer to the photographs as "mug shots." They will also ensure that several formats of photographs are used and that the witness\'s attention is not drawn to any particular photograph. A report must be filed and officers should be cautious before charging a suspect based on mugshots alone. '